# Data preparation

(Consider splitting into train/val/test considering the affordances of the objects instead of randomly, so that e.g. pen, telescope and laptop are in the train set and pencil, microscope and desktop computer in the test set.)


(Also, consider adding multiple images of each object. This way, the model can train on mapping object with its affordances multiple times.)

In [1]:
import pandas as pd

In [2]:
def clean_up(object_name):
    clean_object_name = ''
    for char in object_name:
        if char == '_':
            clean_object_name += ' '
        elif char == '.':
            break
        else:
            clean_object_name += char
    return clean_object_name

In [3]:
file = 'affordance_annotations.txt'
df = pd.read_csv(file)
df.rename(columns = {'Unnamed: 0':'Object'}, inplace = True)
df['Object'] = df['Object'].apply(clean_up)

In [4]:
df

,Object,ImageNet synset,Grasp,Lift,Throw,Push,Fix,Ride,Play,Watch,SitOn,Feed,Row,PourFrom,LookThrough,WriteWith,TypeOn
0,automobile engine,n02761557,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,axe,n02764044,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
2,bicycle,n02834778,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0
3,bottle,n02876657,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0
4,camera,n02942699,1,1,1,1,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,sickle,n04213353,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
58,spoon,n04284002,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
59,stool,n04326896,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
60,typewriter,n04505036,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1


## Approach 1 - pairs of objects and their affordances

In [5]:
df1 = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [7]:
train1 = df1[:42]
val1 = df1[42:52]
test1 = df1[52:]

In [8]:
def get_gold_data_1(table):
    gold_data_pairs = []
    for index, row in table.iterrows():
        for i, value in enumerate(row):
            if type(value) == str:
                pass
            else:
                gold_data_pairs.append((row[0],table.columns[i].lower(),value))
    return gold_data_pairs

In [9]:
train1_pairs = get_gold_data_1(train1)
val1_pairs = get_gold_data_1(val1)
test1_pairs = get_gold_data_1(test1)

In [17]:
train1_pairs

[('hand truck', 'grasp', 0),
 ('hand truck', 'lift', 1),
 ('hand truck', 'throw', 0),
 ('hand truck', 'push', 1),
 ('hand truck', 'fix', 1),
 ('hand truck', 'ride', 1),
 ('hand truck', 'play', 0),
 ('hand truck', 'watch', 0),
 ('hand truck', 'siton', 1),
 ('hand truck', 'feed', 0),
 ('hand truck', 'row', 0),
 ('hand truck', 'pourfrom', 0),
 ('hand truck', 'lookthrough', 0),
 ('hand truck', 'writewith', 0),
 ('hand truck', 'typeon', 0),
 ('serving cart', 'grasp', 0),
 ('serving cart', 'lift', 0),
 ('serving cart', 'throw', 0),
 ('serving cart', 'push', 1),
 ('serving cart', 'fix', 1),
 ('serving cart', 'ride', 1),
 ('serving cart', 'play', 0),
 ('serving cart', 'watch', 0),
 ('serving cart', 'siton', 1),
 ('serving cart', 'feed', 0),
 ('serving cart', 'row', 0),
 ('serving cart', 'pourfrom', 0),
 ('serving cart', 'lookthrough', 0),
 ('serving cart', 'writewith', 0),
 ('serving cart', 'typeon', 0),
 ('automobile engine', 'grasp', 0),
 ('automobile engine', 'lift', 0),
 ('automobile engin

# Approach 2 - pairs of affordances and their objects

In [10]:
def get_gold_data_2(table):
    gold_data_pairs = []
    for index, row in table.iterrows():
        for i, value in enumerate(row):
            if type(value) == str:
                pass
            else:
                gold_data_pairs.append((table.index[0].lower(),table.columns[i].lower(),value))
                
    return gold_data_pairs

In [11]:
df2 = df.transpose()
df2 = df2.sample(frac=1, random_state=42)
df2.columns = df2.iloc[0]

In [13]:
train2 = df2[2:11]
val2 = df2[11:14]
test2 = df2[14:17]

In [14]:
train2

Object,automobile engine,axe,bicycle,bottle,camera,can,car tire,carving knife,chair,chalk,...,monitor,motorcycle,pencil,rhinoceros,serving cart,sickle,spoon,stool,typewriter,walkie-talkie
Push,0,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
WriteWith,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
Feed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
LookThrough,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Play,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PourFrom,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Grasp,0,1,0,1,1,1,0,1,0,1,...,0,0,1,0,0,1,1,0,0,1
Watch,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
TypeOn,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
train2_pairs = get_gold_data_2(train2)
val2_pairs = get_gold_data_2(val2)
test2_pairs = get_gold_data_2(test2)

In [16]:
train2_pairs

[('push', 'automobile engine', 0),
 ('push', 'axe', 1),
 ('push', 'bicycle', 1),
 ('push', 'bottle', 1),
 ('push', 'camera', 1),
 ('push', 'can', 1),
 ('push', 'car tire', 1),
 ('push', 'carving knife', 1),
 ('push', 'chair', 1),
 ('push', 'chalk', 1),
 ('push', 'cleaver', 1),
 ('push', 'desktop computer', 1),
 ('push', 'dish', 1),
 ('push', 'dog', 1),
 ('push', 'dustcloth', 1),
 ('push', 'fishing pole', 1),
 ('push', 'food turner', 1),
 ('push', 'frisbee', 1),
 ('push', 'guitar', 1),
 ('push', 'hand saw', 1),
 ('push', 'handset', 1),
 ('push', 'horse', 0),
 ('push', 'laptop', 1),
 ('push', 'microscope', 1),
 ('push', 'mobile phone', 1),
 ('push', 'mop', 1),
 ('push', 'pen', 1),
 ('push', 'pitcher', 1),
 ('push', 'power saw', 1),
 ('push', 'shopping cart', 1),
 ('push', 'small boat', 1),
 ('push', 'sofa', 0),
 ('push', 'teapot', 1),
 ('push', 'telescope', 1),
 ('push', 'television', 1),
 ('push', 'toothbrush', 1),
 ('push', 'umbrella', 1),
 ('push', 'vacuum cleaner', 1),
 ('push', 'vio